### ANN Regression Practical Implementation

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import pickle

In [8]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Estimated salary will be our output feature, and remaining will be independent features.<br>
Since estimated salary is continuous, this is a regression problem.

In [9]:
#Preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [ ]:
data.head()  #those columns have been dropped

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
#Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [12]:
#One hot encode geography
one_hot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

In [13]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
#Combine One Hot encoded columns with data and dropping geography
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [15]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [16]:
#Split data into features and target
X = data.drop('EstimatedSalary', axis=1)
Y = data['EstimatedSalary']

In [17]:
#Split the data in train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size= 0.2, random_state= 42)

In [18]:
#SCale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [19]:
#Save the scalers and encoders for later use using pickle
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
    
with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo, file)
    
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

#### ANN Regression Problem Statement

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [21]:
#Building the model
model = Sequential([
    Dense(64, activation='relu', input_shape = (X_train.shape[1],)), #Hidden layer 1, input layer connected to it, it refers to the shape(no. of columns of row 1)
    Dense(32, activation='relu'), #Hidden layer 2   
    Dense(1) #Output layer for regression
    #When we dont apply any activation function, it means linear activation function, which is default for Regression
])

#Compile the model
model.compile(optimizer = 'adam', loss= 'mean_absolute_error', metrics = ['mae'])

model.summary()

c:\Users\rohan\GenAIUdemy\NLP\ANN_Project\env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
#logs
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

#Set up Tensorboard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d - %H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq = 1)

In [23]:
#Set up EarlyStopping based on validation loss
early_stop_callback = EarlyStopping(monitor= 'val_loss', patience = 10, restore_best_weights= True) #we will wait for 10 epochs minimum

In [25]:
#Train the model
history = model.fit(
    X_train, Y_train,
    validation_data = (X_test, Y_test),
    epochs = 100,
    callbacks = [early_stop_callback, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 100175.8516 - mae: 100175.8516 - val_loss: 98488.2578 - val_mae: 98488.2578
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 99482.9688 - mae: 99482.9688 - val_loss: 96843.4062 - val_mae: 96843.4062
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 97022.9297 - mae: 97022.9297 - val_loss: 92686.3516 - val_mae: 92686.3516
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 92387.2188 - mae: 92387.2188 - val_loss: 85824.8984 - val_mae: 85824.8984
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 84330.0078 - mae: 84330.0078 - val_loss: 76994.8594 - val_mae: 76994.8594
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 76278.6641 - mae: 76278.6641 - val_loss: 67810.5547 - val_mae: 67810.5547
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 67834.3203 - mae: 67834.3203 - val_loss: 60099.9492 - val_mae: 60099.9492
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step 

In [27]:
%load_ext tensorboard

In [30]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 16760), started 0:00:12 ago. (Use '!kill 16760' to kill it.)

In [33]:
#Evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test, Y_test)
print(f"Test MAE = {test_mae:.2f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51148.7578 - mae: 51148.7578
Test MAE = 50281.95


In [34]:
#Save the model
model.save('regression_model.h5')